<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# Workshop: Microsoft SQL Server Machine Learning Services

#### <i>A Microsoft Course from the SQL Server team</i>

## 06 - Phase 4: Deployment 

<p style="border-bottom: 1px solid lightgrey;"></p>

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/textbubble.png?raw=true"><b>Phase 4 - Deployment</b></p>

<br>
<br>
<br>

You're learning to use the Team Data Science Process to create a complete solution, using SQL Server as the platform. The phases in the Team Data Science process are:

<dl>
  <dt>1 - Business Understanding</dt>
  <dt>2 - Data Acquisition and Understanding</dt>
  <dt>3 - Modeling</i></dt>
  <dt>4 - Deployment <i>(This module)</dt>
  <dt>5 - Customer Acceptance and Model Retraining</dt>
<dl>

<p style="border-bottom: 1px solid lightgrey;"></p>

<img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/pin.jpg?raw=true"><b>4.0 - Deployment Overview</b>
<br>

In this phase you'll take the Trained Model from the Azure environment and then use the Model in SQL Server using ML Services over your data.

### Goal for Deployment
- Deploy models with a data pipeline to a production or production-like environment for final user acceptance

### How to do it
- Deploy the model and pipeline to a production or production-like environment for application consumption


<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/pin.jpg?raw=true"><b>4.1 Load or Create the Model in SQL Server</b></p>

In this section, you will create the Model in SQL Server and deploy your predictive model with the help of SQL Server Machine Learning Services.

To "deploy" a model, you have two options: You can create and store a binary representation of the model in a hosting environment, or you can re-run the code that calls an algorithm and implement a prediction function that uses the model to predict. That function can be called from applications, on a schedule, or an API service. 

To deploy the model so that you can use it in SQL Server, you will need to either copy the model to a location and SQL Server can read, load it into a binary column of a table in the SQL Server database, or train and call the model from T-SQL code using the `sp_execute_external_script` stored procedure. You'll follow that latter process in a moment.

If you are transferring a trained binary model in, you'll use code to create the  binary file *(called a pickle in Python, or a stream in R)* that contains the Model. There are other serialization methods available, including ONNX that you can use to store your model. If you use Azure Machine Learning Services, the Model is stored in the Cloud.

**NOTE: The Model essentially contains the code from the final training. That means that the receiving system (SQL Server in this course) must be able to run that code, have the same libraries and so on.**

If you create the training for the model on a local system, it's important to use as similar data (as much as possible) for the model predictions. The data you trained with needs to represent as closely as possible the type of data you're scoring, or you could get incorrect results. 

In this course, rather than store the model as a binary object, you'll just re-create the R code from the Data Scientist's Notebook, but adapt it for a stored procedure call using the Extensibility Framework you learned about earlier.


<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/checkbox.png?raw=true"><b>Activity: Create a Stored Procedure to load the data and call the Model training</b></p>

You're going to combine the data selection and the call to the Model in one step. This is common in K-Means or other unsupervised learning, but for labeled data its more common to train the Model, store it as a binary object, and then load that for the calls, making two steps instead of the combined single step you see here.

- Run the following code on your Instance:

In [0]:
USE [Analysis];
GO


DROP PROC IF EXISTS generate_customer_return_clusters;
GO

CREATE procedure [dbo].[generate_customer_return_clusters]
AS
/*
  This procedure uses R to classify customers into different groups based on their
  purchase & return history.
*/
BEGIN
	DECLARE @duration FLOAT
	, @instance_name NVARCHAR(100) = @@SERVERNAME
	, @database_name NVARCHAR(128) = db_name()
-- Input query to generate the purchase history & return metrics
	, @input_query NVARCHAR(MAX) = N'
SELECT
  ss_customer_sk AS customer,
  round(CASE WHEN ((orders_count = 0) OR (returns_count IS NULL) OR (orders_count IS NULL) OR ((returns_count / orders_count) IS NULL) ) THEN 0.0 ELSE (cast(returns_count as nchar(10)) / orders_count) END, 7) AS orderRatio,
  round(CASE WHEN ((orders_items = 0) OR(returns_items IS NULL) OR (orders_items IS NULL) OR ((returns_items / orders_items) IS NULL) ) THEN 0.0 ELSE (cast(returns_items as nchar(10)) / orders_items) END, 7) AS itemsRatio,
  round(CASE WHEN ((orders_money = 0) OR (returns_money IS NULL) OR (orders_money IS NULL) OR ((returns_money / orders_money) IS NULL) ) THEN 0.0 ELSE (cast(returns_money as nchar(10)) / orders_money) END, 7) AS monetaryRatio,
  round(CASE WHEN ( returns_count IS NULL                                                                        ) THEN 0.0 ELSE  returns_count                 END, 0) AS frequency

FROM
  (
    SELECT
      ss_customer_sk,
      -- return order ratio
      COUNT(distinct(ss_ticket_number)) AS orders_count,
      -- return ss_item_sk ratio
      COUNT(ss_item_sk) AS orders_items,
      -- return monetary amount ratio
      SUM( ss_net_paid ) AS orders_money
    FROM store_sales s
    GROUP BY ss_customer_sk
  ) orders
  LEFT OUTER JOIN
  (
    SELECT
      sr_customer_sk,
      -- return order ratio
      count(distinct(sr_ticket_number)) as returns_count,
      -- return ss_item_sk ratio
      COUNT(sr_item_sk) as returns_items,
      -- return monetary amount ratio
      SUM( sr_return_amt ) AS returns_money
    FROM store_returns
    GROUP BY sr_customer_sk
  ) returned ON ss_customer_sk=sr_customer_sk
 '
EXEC sp_execute_external_script
	  @language = N'R'
	, @script = N'
# Define the connection string
connStr <- paste("Driver=SQL Server;Server=", instance_name, " ;Database=", database_name, " ;Trusted_Connection=true;", sep="" );

# Input customer data that needs to be classified. This is the result we get from our query
customer_returns <- RxSqlServerData(sqlQuery = input_query,
									colClasses = c(customer = "numeric", orderRatio = "numeric", itemsRatio = "numeric", monetaryRatio = "numeric", frequency = "numeric"),
									connectionString = connStr);

# Output table to hold the customer cluster mappings
return_cluster = RxSqlServerData(table = "customer_return_clusters", connectionString = connStr);

# set.seed for random number generator for predicatability
set.seed(10);

# generate clusters using rxKmeans and output clusters to a table called "customer_return_clusters".
clust <- rxKmeans( ~ orderRatio + itemsRatio + monetaryRatio + frequency, customer_returns, numClusters = 4
                    , outFile = return_cluster, outColName = "cluster", writeModelVars = TRUE , extraVarsToWrite = c("customer"), overwrite = TRUE);
'
	, @input_data_1 = N''
	, @params = N'@instance_name nvarchar(100), @database_name nvarchar(128), @input_query nvarchar(max), @duration float OUTPUT'
	, @instance_name = @instance_name
	, @database_name = @database_name
	, @input_query = @input_query
	, @duration = @duration OUTPUT;
END;

GO

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/pin.jpg?raw=true"><b>4.2 Call the model using production data</b></p>

Everything is now in place for you to call the Stored Procedure that will wait for data as input and then run the model, predicting the outcome. You'll send in the Features (in this case, using a `SELECT` Transact-SQL statement as the input variable).


<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/checkbox.png?raw=true"><b>Activity: Call and view the Predictions</b></p>

- Run the code below. The first step calls the Clustering algorithm from the last step, and the second part shows the results:

In [0]:
USE [Analysis];
GO

--Empty table of the results before running the stored procedure
TRUNCATE TABLE customer_return_clusters;
--Execute the clustering. This will load the table customer_return_clusters with cluster mappings
EXEC [dbo].[generate_customer_return_clusters];

--Now select data from table customer_return_clusters to verify that the clustering data was loaded
SELECT * FROM customer_return_clusters;

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/thinking.jpg?raw=true"><b>For Further Study</b></p>

<br>
<br>

 - Another example of using a Stored Procedure to predict data using R and SQL Server: https://docs.microsoft.com/en-us/sql/advanced-analytics/tutorials/rtsql-create-a-predictive-model-r?view=sql-server-2017


<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/education1.png?raw=true"><b>Next</b>: Phase 5 - Customer Acceptance and Model Retraining</p>

Next, you'll continue working through the Team Data Science Process in the final phase - *07 - Phase 5 - Customer Acceptance and Model Retraining*. Open that Notebook to continue.